## import packages and set options

In [1]:
# import necessary packages
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import statsmodels as sm

In [2]:
# set display options
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.2f}'.format
pd.set_option("display.max_rows", 150)


## data input and cleansing

In [5]:
df = pd.read_csv('./stock-market-data.csv')
df[['symbol', 'industry']] = df[['symbol', 'industry']].apply(
    lambda x: x.astype("string"))
df.head()

,symbol,date,pre_close,open,high,low,close,volume,amount,adj_factor,capt,index_w50,index_w300,index_w500,industry
0,600000.SH,20120104,8.49,8.54,8.56,8.39,8.41,34201379,290229551.00,6.66,125500555680.00,0.05,0.02,0.00,BANKS
1,600000.SH,20120105,8.41,8.47,8.82,8.47,8.65,132116203,1144753023.00,6.66,129082022192.00,0.05,0.02,0.00,BANKS
2,600000.SH,20120106,8.65,8.63,8.78,8.62,8.71,61778687,537043761.00,6.66,129977388820.00,0.05,0.02,0.00,BANKS
3,600000.SH,20120109,8.71,8.72,8.99,8.68,8.95,80136249,711429611.00,6.66,133558855331.00,0.05,0.02,0.00,BANKS
4,600000.SH,20120110,8.95,8.95,9.10,8.88,9.07,72004632,647206633.00,6.66,135349588587.00,0.05,0.02,0.00,BANKS


1. 哪些股票的代码中包含"8"这个数字？

In [7]:
# use list comprehension
[ s for s in set(df['symbol']) if s.find('8')>=0][:6]


['300280.SZ', '600588.SH', '000861.SZ', '300238.SZ', '600782.SH', '600785.SH']

2. 每天上涨和下跌的股票各有多少？

In [13]:
# use np.select
conditions = [
    df['close'] > df['pre_close'],
    df['close'] < df['pre_close']]

results = ['UP', 'DOWN']

df['updown'] = np.select(conditions, results, default='STEADY')
df.groupby(['date', 'updown']).agg(num = ('symbol','nunique')).reset_index().head()


,date,updown,num
0,20120104,DOWN,2007
1,20120104,STEADY,122
2,20120104,UP,191
3,20120105,DOWN,2071
4,20120105,STEADY,117


3. 每天每个交易所上涨、下跌的股票各有多少？

In [15]:
# based on question 2

df['exchange'] = df['symbol'].str[-2:]
df.groupby(['date', 'exchange','updown']).agg(num = ('symbol','nunique')).reset_index().head()


,date,exchange,updown,num
0,20120104,SH,DOWN,794
1,20120104,SH,STEADY,42
2,20120104,SH,UP,85
3,20120104,SZ,DOWN,1213
4,20120104,SZ,STEADY,80


4. 沪深300成分股中，每天上涨、下跌的股票各有多少？

In [16]:
# based on question 2
df.query("index_w300 > 0").groupby(['date', 'Flag']).agg(num = ('symbol','nunique')).reset_index().head()

,date,Flag,num
0,20120104,DOWN,275
1,20120104,STEADY,5
2,20120104,UP,20
3,20120105,DOWN,242
4,20120105,STEADY,8


5. 每天每个行业各有多少只股票？

In [17]:
# based on question 2
df.groupby(['date', 'industry']).agg(num = ('symbol','nunique')).reset_index().head()


,date,industry,num
0,20120104,AERODEF,10
1,20120104,AIRLINE,12
2,20120104,AUTO,85
3,20120104,BANKS,16
4,20120104,BEV,30


6. 股票数最大的行业和总成交额最大的行业是否总是同一个行业？

In [18]:
# get # of stocks and $ of amount by industry
df1 = df.groupby(['date', 'industry']).agg({'symbol': 'nunique', 'amount': 'sum'}).reset_index()
df1.head()


,date,industry,symbol,amount
0,20120104,AERODEF,10,493331236.30
1,20120104,AIRLINE,12,359576492.30
2,20120104,AUTO,85,2299263904.60
3,20120104,BANKS,16,3612012715.00
4,20120104,BEV,30,2946962799.60
